# Logistic Regression


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import utils as ml
from logisticClassify import *

In [12]:
iris = np.genfromtxt("data/iris.txt", delimiter=None)
X, Y = iris[:, 0:2], iris[:, -1]  # get first two features & target
X, Y = ml.shuffleData(X, Y)  # reorder randomly (important later)
X, _ = ml.rescale(X)  # works much better on rescaled data

XA, YA = X[Y < 2, :], Y[Y < 2]  # get class 0 vs 1
XB, YB = X[Y > 0, :], Y[Y > 0]  # get class 1 vs 2

print(XA.shape, YA.shape)
print(XB.shape, YB.shape)

(99, 2) (99,)
(99, 2) (99,)
